# Creating the field

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import mean_squared_error as mse
from sklearn.cross_validation import train_test_split
import time
import sys
import time
import pickle

/home/lisov/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
from scipy import optimize as opt
from itertools import combinations_with_replacement as cwr

In [15]:
num_of_smpls = 231
X_full = np.array([])

for item in cwr([0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 
                 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0], 2):
    X_full = np.append(X_full, item)
X_full = X_full.reshape(231, 2)

y_full = np.zeros(num_of_smpls)
for i in range(len(X_full)):
    y_full[i] = opt.rosen(X_full[i])

X_train, X_test, y_train, y_test = train_test_split(X_full, y_full, 
                                                    test_size=0.5, random_state=322)

y_train = y_train[:, None]
y_test  =  y_test[:, None]
#print(y_train)
print(f'Train shapes:{X_train.shape, y_train.shape}')
print(f'Test shapes:{X_test.shape, y_test.shape}')

Train shapes:((115, 2), (115, 1))
Test shapes:((116, 2), (116, 1))


# Preparing for the NN

In [9]:
ndim = X_full.shape[1] #dimention
layers = [256,128,128,128]

learning_rate_decay = .97
start_learning_rate = 3e-4
learning_rate_schedule_epochs = 2000

In [10]:
tf.reset_default_graph()  #Clears the default graph stack and resets the global default graph.

# placeholders
x = tf.placeholder(tf.float32, [None, ndim]) #Inserts a placeholder for a tensor that will be always fed.
y_ = tf.placeholder(tf.float32, [None, 1])

learning_rate_ = tf.placeholder(tf.float32)
forces_coeff_ = tf.placeholder(tf.float32)
keep_probability_ = tf.placeholder(tf.float32, name='keep_probability')
l2_reg_ = tf.placeholder(tf.float32, name='l2reg')

# weights
W1 = tf.Variable(tf.truncated_normal([ndim, layers[0]], stddev=(2/ndim)**.5))
b1 = tf.Variable(tf.truncated_normal([layers[0]],  stddev=.1))
h1 = tf.nn.relu(tf.matmul(x, W1) + b1)
h_drop1 = tf.nn.dropout(h1, keep_probability_, noise_shape = [1,layers[0]])
Ws = [W1]; bs = [b1]; hs = [h_drop1]
for cnt_layer in range(1, len(layers)):
    Ws.append(tf.Variable(tf.truncated_normal([layers[cnt_layer - 1], layers[cnt_layer]], 
                                              stddev=(2/layers[cnt_layer - 1])**.5)))
    bs.append(tf.Variable(tf.truncated_normal([layers[cnt_layer]],  stddev=.1))) 
    hs.append(tf.nn.dropout(tf.nn.relu(tf.matmul(hs[-1], Ws[-1]) + bs[-1]), keep_probability_,
                            noise_shape = [1,layers[cnt_layer]])) # noise shape is important
Ws.append(tf.Variable(tf.truncated_normal([layers[-1], 1], stddev=.1)))
bs.append(tf.Variable(tf.truncated_normal([1],  stddev=.1)))

# losses and outputs
y = tf.matmul(hs[-1], Ws[-1]) + bs[-1]
l2_regularizer = sum(tf.nn.l2_loss(Wxxx) for Wxxx in Ws) 
mse_e = tf.losses.mean_squared_error(predictions = y, labels = y_)
loss = mse_e + l2_reg_*l2_regularizer

# some extra stuff for learning rate decay
global_step = tf.Variable(0, trainable=False)
starter_learning_rate = start_learning_rate
learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           learning_rate_schedule_epochs, learning_rate_decay, staircase=True)
#Applies exponential decay to the learning rate.

lr_fun = lambda: learning_rate
min_lr = lambda: tf.constant(1e-5)
actual_lr = tf.case([(tf.less(learning_rate, tf.constant(1e-5)), min_lr)], default=lr_fun)

train_step = tf.train.AdamOptimizer(learning_rate=actual_lr).minimize(loss, global_step=global_step)

# Functions

In [11]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=True): # we need this to iterate over data
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

def get_errors(x_, y_): # return rmse, mae, maxae
    return [np.sqrt(mse(x_, y_)), np.mean(np.abs(x_ - y_)), np.max(np.abs(x_ - y_))]

def get_mcdues(X): # returns MCD UEs
    stds = np.zeros((X.shape[0], T), dtype = float)
    for cnt_ in range(T):
        stds[:, cnt_] = np.ravel(sess.run(y, feed_dict={x: X, 
                                                        keep_probability_: .5}))
    return np.std(stds, axis = 1)

# Creating a NN

In [12]:
try:
    sess.close()
except:
    pass

config = tf.ConfigProto()
config.gpu_options.allow_growth = True

init = tf.global_variables_initializer()
saver = tf.train.Saver()
sess = tf.Session(config = config)
sess.run(init)
epoch = 0
data = []

In [13]:
batch_size = 30 # please note that I used GPU and maybe you want to decrease the batch size to 200 or so
init_epochs = 5000
keep_probs = [.5, .75, .9]
l2_reg = 1e-4
check_step = 500

T = 25

In [14]:
t = time.time()
for keep_prob in keep_probs:
    for cnt in range(init_epochs):
        epoch += 1

        for batch in iterate_minibatches(X_train, y_train, batch_size):
            X_batch, y_batch = batch
            sess.run(train_step, feed_dict={x: X_batch, 
                                            y_: y_batch, 
                                            keep_probability_: keep_prob, 
                                            l2_reg_: l2_reg})
        if (epoch+1) % check_step == 0:
            print(np.round(time.time() - t, 2), end='s')
            t = time.time()
            preds_train = sess.run(y, feed_dict={x: X_train, keep_probability_: 1})
            preds_test = sess.run(y, feed_dict= {x: X_test , keep_probability_: 1})

            train_err =  get_errors(preds_train, y_train)
            test_err =  get_errors(preds_test, y_test)
            print(' &', np.round(time.time() - t, 2), 's')
            print(epoch, np.round(train_err, 4), np.round(test_err, 4), end = '|')

2.01s & 0.02 s
499 [ 46.1602  26.5313 346.0827] [ 42.5472  28.403  241.3903]|

KeyboardInterrupt: 

# MCD UE showcase

In [10]:
preds_test = sess.run(y, feed_dict= {x: X_test , keep_probability_: 1})
errors = np.abs(preds_test - y_test)

In [11]:
def get_assessed_val(preds_test, X_test):
    m = preds_test
    s = get_mcdues
    return (m, s)

# GPyOpt_BO using output of NN

In [17]:
%pylab inline
import GPyOpt
import GPy
from scipy import optimize as opt

Populating the interactive namespace from numpy and matplotlib


In [18]:
#func = opt.rosen

def rosenbrock_for_optimization(x):
    return sum(100.0*(x[1:] - x[:-1]**2.0)**2.0 + (1 - x[:-1])**2.0)

func = GPyOpt.objective_examples.experiments2d.rosenbrock()

objective = GPyOpt.core.task.SingleObjective(func.f)

space = GPyOpt.Design_space(space =[{'name': 'var_1', 'type': 'discrete', 'domain': (-5, 5)},
                                    {'name': 'var_2', 'type': 'discrete', 'domain': (-5, 5)}])

model = GPyOpt.models.GPModel(optimize_restarts=5,verbose=False)

aquisition_optimizer = GPyOpt.optimization.AcquisitionOptimizer(space, optimizer='DIRECT')

initial_design = GPyOpt.experiment_design.initial_design('random', space, 5)

In [19]:
from GPyOpt.acquisitions.base import AcquisitionBase
from GPyOpt.acquisitions.EI import AcquisitionEI
from numpy.random import beta
from scipy.special import erfc

class Exp_impr(AcquisitionBase):
    
    analytical_gradient_prediction = False
   
    def __init__(self, model, space, optimizer=None, cost_withGradients=None, par_a=1, par_b=1, num_samples= 10):
        super(Exp_impr, self).__init__(model, space, optimizer)
        
        self.par_a = par_a
        self.par_b = par_b
        self.num_samples = num_samples
        self.samples = beta(self.par_a,self.par_b,self.num_samples)
        self.EI = AcquisitionEI(model, space, optimizer, cost_withGradients)
        
    def compute_exp_impr(self, x):
        jitter = self.EI.jitter
        #m, s = self.model.predict(x)
        m, s = get_assessed_val(preds, X_test)
        fmin = self.model.get_fmin()
        phi, Phi, u = self.get_quantiles(jitter, fmin, m, s)
        f_acqu = s * (u * Phi + phi)
        cost_x, _ = self.cost_withGradients(x)
        return -(f_acqu*self.space.indicator_constraints(x))/cost_x
    
    
    def get_quantiles(self, acquisition_par, fmin, m, s):
    
        if isinstance(s, np.ndarray):
            s[s<1e-10] = 1e-10
        elif s< 1e-10:
            s = 1e-10
        u = (fmin - m - acquisition_par)/s
        phi = np.exp(-0.5 * u**2) / np.sqrt(2*np.pi)
        Phi = 0.5 * erfc(-u / np.sqrt(2))
        return (phi, Phi, u)
    
    def acquisition_function(self,x):
        acqu_x = np.zeros((x.shape[0],1))       
        for k in range(self.num_samples):
            self.EI.jitter = self.samples[k]
            acqu_x +=self.compute_exp_impr(x) #here          
        return acqu_x/self.num_samples

In [20]:
acquisition = Exp_impr(model, space, optimizer=aquisition_optimizer, par_a=1, par_b=10, num_samples=200)
evaluator = GPyOpt.core.evaluators.Sequential(acquisition)

bo = GPyOpt.methods.ModularBayesianOptimization(model, space, objective, acquisition, evaluator, initial_design)

In [21]:
bo.space

In [22]:
max_iter  = 10
eps       = 1e-6     ## tolerance, max distance between consicutive evaluations.

bo.run_optimization(max_iter, eps) 

AttributeError: 'OptDirect' object has no attribute 'space'

In [23]:
print(bo.x_opt)
print(bo.fx_opt)

AttributeError: 'ModularBayesianOptimization' object has no attribute 'x_opt'

In [272]:
bo.plot_acquisition()
bo.plot_convergence()

TypeError: '<' not supported between instances of 'function' and 'float'